## 라이브러리 로드

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

# 번역
from googletrans import Translator
# 거리계산
from haversine import haversine, Unit
# 상관분석
import pingouin as pg
# 회귀분석
from statsmodels.formula.api import ols

# pandas 보기 설정
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)

# 그래프에 한글 적용을 위한 설정
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

In [8]:
# pip install --upgrade pingouin

### 데이터 불러오기

In [15]:
orders = pd.read_csv("olist_orders_dataset.csv")
customer = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
items = pd.read_csv("olist_order_items_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")

### 지리데이터 정리하기

In [20]:
geolocation.sort_values(by="geolocation_zip_code_prefix").head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
1246,1001,-23.54929,-46.63356,sao paulo,SP
429,1001,-23.55050,-46.63434,sao paulo,SP
1182,1001,-23.54978,-46.63396,sao paulo,SP
1435,1001,-23.54929,-46.63356,sao paulo,SP
326,1001,-23.55143,-46.63407,sao paulo,SP


```
zip_code 는 같은데 위경도가 다른 경우가 많다. 완전한 zip_code 가 아니라 첫 부분만 표기 된 상태이기 때문이다.
이로인해 customer 와 seller 의 정확한 거리 계산은 불가능 한 상황이다.
zip_code 별로 중복제거가 필요한 상황이지만 단순 중복제거를 하면 첫번째 값이 남기 때문에 평균을 내서 적용해보려고 한다.
```

In [24]:
geolocation = geolocation.groupby("geolocation_zip_code_prefix").mean()
geolocation

,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.54929,-46.63356
1002,-23.54832,-46.63542
1003,-23.54903,-46.63531
1004,-23.55012,-46.63512
1005,-23.54982,-46.63561
...,...,...
99960,-27.95380,-52.02964
99965,-28.17389,-52.03845
99970,-28.34514,-51.87693


### seller 와 cumstomer 데이터에 geo 정보 합치기

In [26]:
sellers = sellers.merge(geolocation, left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state,geolocation_lat,geolocation_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,-22.89854,-47.06313
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,-22.38294,-46.94664
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,-22.91064,-43.17651
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,-23.65725,-46.61076
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,-22.97165,-46.53362


In [27]:
customer = customer.merge(geolocation, left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")
customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.50990,-47.39787
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.72685,-46.54575
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.52779,-46.66031
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,-23.49693,-46.18535
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,-22.98722,-47.15107


- 테이블을 합쳤을때, customer 와 seller 의 geo 정보가 헷갈리지 않도록 컬럼명을 rename 해준다. 

In [35]:
customer = customer.rename(columns={"geolocation_lat":"customer_lat", "geolocation_lng":"customer_lng"})
customer.sample(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng
67370,0294d91f6bcc29bc9c30fae5872430d0,63dba940a39a61871dfdb9b529c1582d,8574,itaquaquecetuba,SP,-23.50583,-46.36204
77907,710dd4bda0d8cbfb51e101c8085eea04,3d39a9d77c73f1e6b8c49b3dc5c956cc,13087,campinas,SP,-22.85276,-47.05510


In [36]:
sellers = sellers.rename(columns={"geolocation_lat":"sellers_lat", "geolocation_lng":"sellers_lng"})
sellers.sample(2)

,seller_id,seller_zip_code_prefix,seller_city,seller_state,sellers_lat,sellers_lng
761,b6f52e346160519c24a23af05410ad32,1254,sao paulo,SP,-23.54197,-46.68191
891,458c6fa920810ce2492da71bc26d9eec,15813,catanduva,SP,-21.12022,-49.03876


### 데이터 합치기

In [40]:
df = orders.merge(items, on='order_id', how='left')
df = df.merge(customer, on='customer_id', how='left')
df = df.merge(payments, on='order_id', how='left')
df = df.merge(products, on='product_id', how='left')
df = df.merge(sellers, on = 'seller_id', how='left')
df.shape

(118434, 37)

In [38]:
df.sample(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,sellers_lat,sellers_lng
79249,86ddb141a4b75a77c2ed5893b02ee1f1,f2f6ed8d98af87661950716d6c148b80,delivered,2018-03-22 09:50:09,2018-03-22 10:09:15,2018-03-23 19:08:59,2018-04-04 10:42:55,2018-04-10 00:00:00,1.00000,2052b3c5e08893e567c2fb7778476c75,be8e909810184b9b19e886129149d8e5,2018-03-28 10:09:15,44.99000,13.71000,a679b405524af5adf6fca7ab98607edd,19600,rancharia,SP,-22.22842,-50.90429,1.00000,credit_card,1.00000,58.70000,informatica_acessorios,46.00000,227.00000,1.00000,810.00000,16.00000,10.00000,12.00000,13418.00000,piracicaba,SP,-22.72880,-47.63174
82933,56bc98e6d5b88c2cdb905f2fbec2ca3a,1ac30d3d231e7b32f3c8b1f4770f451d,delivered,2017-07-28 10:51:21,2017-07-28 11:05:19,2017-08-04 14:23:53,2017-08-11 21:57:53,2017-08-21 00:00:00,1.00000,418c8127e2810370323241e326874320,835f0f7810c76831d6c7d24c7a646d4d,2017-08-03 11:05:19,139.90000,15.73000,7cee328bcc37d1d72cbd46ed95dad5e0,35681,itauna,MG,-20.09156,-44.61498,1.00000,credit_card,5.00000,155.63000,fashion_roupa_masculina,44.00000,412.00000,2.00000,300.00000,35.00000,30.00000,20.00000,8030.00000,sao paulo,SP,-23.49997,-46.42366
33326,9f7d811e4738ca24d33c0dc98d2cba67,57353df37982fb517dc5b6249de5cbb3,delivered,2018-08-12 19:14:57,2018-08-12 19:25:13,2018-08-13 15:24:00,2018-08-17 21:04:22,2018-09-06 00:00:00,1.00000,2cc9fe0726e28bee40b04417a8971b53,c3cfdc648177fdbbbb35635a37472c53,2018-08-14 19:25:13,245.00000,32.59000,5492ec0f39ac91306b5dc46fc99ddc95,36015,juiz de fora,MG,-21.75965,-43.35320,1.00000,debit_card,1.00000,277.59000,esporte_lazer,46.00000,1071.00000,1.00000,2600.00000,30.00000,30.00000,20.00000,80610.00000,curitiba,PR,-25.46921,-49.29394


### 컬럼명 변경하기
- 직관적으로 이해하기 쉽게 한글로 컬럼명을 변경한다.

In [41]:
df = df.rename(columns={"order_purchase_timestamp":"고객구매시간"
                   , "order_approved_at":"지불승인시간"
                   , "order_delivered_carrier_date":"물류팀에전달시간"
                   , "order_delivered_customer_date":"실제배송날짜"
                   , "order_estimated_delivery_date":"예상배송날짜"
                   , "shipping_limit_date":"판매자배송날짜"})
df.sample(3)

,order_id,customer_id,order_status,고객구매시간,지불승인시간,물류팀에전달시간,실제배송날짜,예상배송날짜,order_item_id,product_id,seller_id,판매자배송날짜,price,freight_value,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,customer_lat,customer_lng,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,sellers_lat,sellers_lng
17231,25735b38f376510aa1e9762e25343124,bb514cf4c072f5b604be110df9585d95,delivered,2017-05-24 09:46:11,2017-05-24 10:02:31,2017-05-25 10:03:42,2017-05-26 16:13:44,2017-06-06 00:00:00,1.00000,491a99b628dd0701b0758ec05b49cafb,20d83f3ef0e6925fd74bfd59170babf7,2017-05-30 10:02:31,59.90000,7.78000,e58c0f78a7e98fbed6df464265855d37,13275,valinhos,SP,-22.98656,-46.98872,1.00000,credit_card,3.00000,67.68000,fashion_bolsas_e_acessorios,45.00000,809.00000,1.00000,100.00000,16.00000,5.00000,16.00000,2804.00000,sao paulo,SP,-23.48381,-46.69528
78941,6a2bb0f0f146e59e7b2669959798670e,e8b0ff75e9847ee1b34fddb882b41b85,delivered,2018-05-16 13:16:35,2018-05-16 13:33:39,2018-05-17 14:10:00,2018-05-22 17:46:27,2018-06-04 00:00:00,1.00000,173e9fe34bfe97f3a5e6dc57fe897b74,ba143b05f0110f0dc71ad71b4466ce92,2018-05-21 13:30:26,57.89000,16.32000,a7a054aada3cfdef011a637e301ac87f,35171,coronel fabriciano,MG,-19.53080,-42.57081,1.00000,credit_card,3.00000,74.21000,livros_tecnicos,36.00000,890.00000,1.00000,950.00000,29.00000,3.00000,21.00000,2274.00000,sao paulo,SP,-23.46824,-46.58503
43275,414b227d34e12d2c95093f0bdb4c44bd,c05135e931996796d1b5bff60251825a,delivered,2018-05-18 11:04:32,2018-05-18 13:07:00,2018-05-21 16:43:00,2018-05-25 13:30:58,2018-06-07 00:00:00,1.00000,06edb72f1e0c64b14c5b79353f7abea3,391fc6631aebcf3004804e51b40bcf1e,2018-05-23 13:07:00,39.99000,17.14000,a2027a1c6f9e04a875c44ea09061d3f2,2212,sao paulo,SP,-23.49453,-46.58841,1.00000,credit_card,6.00000,164.16000,cama_mesa_banho,39.00000,161.00000,2.00000,350.00000,25.00000,8.00000,19.00000,14940.00000,ibitinga,SP,-21.76648,-48.83155


### 기본정보 확인

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118434 entries, 0 to 118433
Data columns (total 37 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    118434 non-null  object 
 1   customer_id                 118434 non-null  object 
 2   order_status                118434 non-null  object 
 3   고객구매시간                      118434 non-null  object 
 4   지불승인시간                      118258 non-null  object 
 5   물류팀에전달시간                    116360 non-null  object 
 6   실제배송날짜                      115037 non-null  object 
 7   예상배송날짜                      118434 non-null  object 
 8   order_item_id               117604 non-null  float64
 9   product_id                  117604 non-null  object 
 10  seller_id                   117604 non-null  object 
 11  판매자배송날짜                     117604 non-null  object 
 12  price                       117604 non-null  float64
 13  freight_value 

- 시간에 대한 컬럼들이 object 타입으로 되어있어서 시간에 대한 type 으로 변경하자. 

In [47]:
df['고객구매시간'] = pd.to_datetime(df['고객구매시간'])
df['지불승인시간'] = pd.to_datetime(df['지불승인시간'])
df['물류팀에전달시간'] = pd.to_datetime(df['물류팀에전달시간'])
df['실제배송날짜'] = pd.to_datetime(df['실제배송날짜'])
df['예상배송날짜'] = pd.to_datetime(df['예상배송날짜'])
df['판매자배송날짜'] = pd.to_datetime(df['판매자배송날짜'])

### 기간 파생변수 추가

In [49]:
df["예상배송기간"] = df["예상배송날짜"] - df["고객구매시간"]
df["결제후내부처리시간"] = df["물류팀에전달시간"] - df["지불승인시간"]
df["물류팀-배송까지기간"] = df["실제배송날짜"] - df["물류팀에전달시간"]
df["실제배송시간"] = df["실제배송날짜"] - df["고객구매시간"]
df["예상보다빨리온날짜"] = df["실제배송날짜"] - df["예상배송날짜"]

In [50]:
df["예상보다빨리온날짜"]

0         -8 days +21:25:13
1         -8 days +21:25:13
2         -8 days +21:25:13
3         -6 days +15:27:45
4        -18 days +18:06:29
                ...        
118429    -2 days +17:37:56
118430    -6 days +11:24:17
118431   -21 days +23:32:54
118432   -21 days +23:32:54
118433   -18 days +13:08:30
Name: 예상보다빨리온날짜, Length: 118434, dtype: timedelta64[ns]

- timedelta64 타입을 숫자형으로 변경

In [53]:
df["예상배송기간"] = df["예상배송기간"].dt.total_seconds() / (60 * 60 * 24)
df["결제후내부처리시간"] = df["결제후내부처리시간"].dt.total_seconds() / (60 * 60 * 24)
df["물류팀-배송까지기간"] = df["물류팀-배송까지기간"].dt.total_seconds() / (60 * 60 * 24)
df["실제배송시간"] = df["실제배송시간"].dt.total_seconds() / (60 * 60 * 24)
df["예상보다빨리온날짜"] = df["예상보다빨리온날짜"].dt.total_seconds() / (60 * 60 * 24)

In [55]:
df["예상보다빨리온날짜"]

0         -7.10749
1         -7.10749
2         -7.10749
3         -5.35573
4        -17.24550
            ...   
118429    -1.26532
118430    -5.52480
118431   -20.01882
118432   -20.01882
118433   -17.45243
Name: 예상보다빨리온날짜, Length: 118434, dtype: float64

In [46]:
df.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
고객구매시간                           0
지불승인시간                         176
물류팀에전달시간                      2074
실제배송날짜                        3397
예상배송날짜                           0
order_item_id                  830
product_id                     830
seller_id                      830
판매자배송날짜                        830
price                          830
freight_value                  830
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
customer_lat                   317
customer_lng                   317
payment_sequential               3
payment_type                     3
payment_installments             3
payment_value                    3
product_category_name         2528
product_name_lenght           2528
product_description_lenght    2528
product_photos_qty            2528
product_weight_g    